## ETL Run Notebook  - Run scripts & Test w/ sql queries

### Run create_table.py to see if tables can be created property

- Make sure you open a RedShift cluster and make it publicly accessible

In [1]:
import psycopg2 

try:
    %run create_table.py 
except Exception as e:
    print(e)

CREATE TABLE IF NOT EXISTS naturaldisaster_stage(
        naturaldisaster_id INT IDENTITY(1, 1),
        year int, 
        seq int, 
        glide varchar,
        disaster_group varchar,
        disaster_subgroup varchar,
        disaster_type varchar, 
        disaster_subtype varchar,
        disaster_subsubtype varchar,
        event_name varchar,
        country varchar,
        iso varchar,
        region varchar,
        continent varchar,
        location varchar(5000),
        origin varchar, 
        associated_dis varchar,
        associated_dis2 varchar,
        OFDA_response varchar, 
        appeal varchar,
        declaration varchar,
        aid_contribution FLOAT,
        dis_mag_value FLOAT,
        dis_mag_scale varchar,
        latitude varchar,
        longitude varchar,
        local_time varchar,
        river_basin varchar(1000),
        start_year BIGINT,
        start_month varchar,
        start_day int,
        end_year BIGINT,
        end_month varchar,
  

### etl.py 
- copies data into staging table 
- inserts data from staging table to final tables 

In [2]:
try:
    %run etl.py 
    
except Exception as e:
    print(e)
    


### dataquality.py 
- Checks if all rows have been entered 
- Checks if all columns have been entered ( only population & co2emission tables have the same number of columns) 
- Check if Disaster Group or Disaster Type have any nulls 
- Check ratio of nulls in total_dealth column, as that column could be important for analysis 

In [1]:
import psycopg2 
try: 
    %run dataquality.py
except Exception as e:
    print(e)

All rows of naturaldisaster have been entered
All rows of surface temp have been entered
All rows of population have been entered
All columns of population have been entered
All rows of co2emissions have been entered
All columns of co2emissions have been entered
Disaster Group contains no Nulls
Disaster Type contains no Nulls
Percentage of Null in total_death column is equal to input


#### Test tables with some Queries

In [11]:
import pandas as pd
import boto3
import json
import psycopg2

In [12]:
config_conn = configparser.ConfigParser()
config_conn.read('dwh.cfg')

# Run a query

query = ('''
SELECT * FROM surfacetemp
''')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config_conn['CLUSTER'].values()))
df = pd.read_sql_query(query, conn)


df.head()

,year,month,country,avg_temperature,avg_temperature_uncertainty
0,1748,4,Åland,NaN,NaN
1,1751,12,Åland,NaN,NaN
2,1755,4,Åland,2.818,4.353
3,1762,12,Åland,-0.722,5.937
4,1766,4,Åland,4.707,4.069


In [10]:
query = ('''
SELECT country, count(*) as num
FROM naturaldisaster_info
GROUP BY country
Order by num desc
''')
df_disnum = pd.read_sql_query(query, conn)

df_disnum

,country,num
0,United States of America (the),1087
1,China,980
2,India,752
3,Philippines (the),668
4,Indonesia,572
...,...,...
223,Bahrain,1
224,United Arab Emirates (the),1
225,Brunei Darussalam,1
226,Qatar,1


In [38]:
query = ('''
SELECT pp.country, pp.population, di.disaster_group
FROM population pp
Join naturaldisaster_info di on pp.country = di.country
WHERE pp.year = '2000'
Order by pp.population
''')
df_popdis = pd.read_sql_query(query, conn)

df_popdis

,country,population,disaster_group
0,Tuvalu,9394,Natural
1,Tuvalu,9394,Natural
2,Tuvalu,9394,Natural
3,Tuvalu,9394,Natural
4,Tuvalu,9394,Natural
...,...,...,...
11584,China,1262645000,Natural
11585,China,1262645000,Natural
11586,China,1262645000,Natural
11587,China,1262645000,Natural


In [ ]:
conn.close()